In [3]:
import pandas as pd 

data  = pd.read_csv('source/jobs.csv')
data.columns = ['index', 'context']
data.head()

,index,context
0,0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo..."
1,1,"{""@context"": ""http://schema.org"", ""@type"": ""Jo..."
2,2,"{""@context"": ""http://schema.org"", ""@type"": ""Jo..."
3,3,"{""@context"": ""http://schema.org"", ""@type"": ""Jo..."
4,4,"{""@context"": ""http://schema.org"", ""@type"": ""Jo..."


In [4]:
data.dropna(axis=0, inplace=True)

In [5]:
data.duplicated().sum()

0

In [7]:
import os

for i, row in data.iterrows():
    with open(os.path.join('my', 'extract', f'{i}.txt'), 'w') as f:
        f.write(row['context'])
    f.close()

In [50]:
template = {
    "job": {
        "title": None,
        "industry": None,
        "description": None,
        "employment_type": None,
        "date_posted": None,
    },
    "company": {
        "name": None,
        "link": None,
    },
    "education": {
        "required_credential": None,
    },
    "experience": {
        "months_of_experience": None,
        "seniority_level": None,
    },
    "salary": {
        "currency": None,
        "min_value": None,
        "max_value": None,
        "unit": None,
    },
    "location": {
        "country": None,
        "locality": None,
        "region": None,
        "postal_code": None,
        "street_address": None,
        "latitude": None,
        "longitude": None,
    },
}

In [9]:
import html

def text_decoder(text):
    decoded_text = html.unescape(text)
    utf8_text = decoded_text.encode('utf-8')

    return utf8_text.decode('utf-8')

In [12]:
import json 

data_as_json = []
for file in os.listdir(os.path.join('my', 'extract')):
    with open(os.path.join('my', 'extract', file), 'r') as f:
        # Load the JSON data from the file
        data = json.load(f)
        for key in data.keys():
            if isinstance(data[key], str):
                data[key] = text_decoder(data[key])
        data_as_json.append(data)
    f.close()

In [13]:
data_as_json[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-08-25T17:45:15.000Z',
 'description': "<p><strong><u>ABOUT US:</u></strong></p><p>As the leader of rehabilitative care, Encompass Health offers both facility-based and home-based patient care through its national network of rehabilitation hospitals, home health agencies, and hospice agencies that spans 42 states and Puerto Rico. Setting the standard for providing excellent care, Encompass Health has earned its place among Modern Healthcare’s “Best Places to Work in Healthcare” and the Fortune “World’s Most Admired Companies” for 2021.</p><p><br></p><p><strong><u>POSITION PURPOSE:</u></strong></p><p>The&nbsp;<strong>HRIS Security &amp; Data Maintenance Analyst</strong>&nbsp;plays an integral role in HR Systems security definition and is responsible for defining, setting up, maintaining and accurately documenting security access for all HR systems. The HRIS Security &amp; Data Maintenance Analyst also oversees

In [45]:
job_mapping = {
    'date_posted': 'datePosted',
    'description': 'description',
    'employment_type': 'employmentType',
    'industry': 'industry',
    'title': 'title'
}
location_mapping = {
    'country': 'jobLocation.address.addressCountry',
    'latitude': 'jobLocation.latitude',
    'locality': 'jobLocation.address.addressLocality',
    'longitude': 'jobLocation.longitude',
    'postal_code': 'jobLocation.address.postalCode',
    'region': 'jobLocation.address.addressRegion',
    'street_address': 'jobLocation.address.streetAddress',
}
company_mapping = {
    'link': 'hiringOrganization.sameAs',
    'name': 'hiringOrganization.name',
}
salary_mapping = {
    'currency': 'estimatedSalary.currency',
    'max_value': 'estimatedSalary.value.maxValue',
    'min_value': 'estimatedSalary.value.minValue',
    'unit': 'estimatedSalary.value.unitText',
}
for data in data_as_json:
    for key_in_template, key_in_data in job_mapping.items():
        if key_in_data in data.keys():
            template['job'][key_in_template] = data[key_in_data]

    for key_in_template, key_in_data in company_mapping.items():
        if key_in_data in data.keys():
            keys = key_in_data.split('.')
            value = data
            for key in keys:
                if key in value:
                    value = value[key]
                else:
                    break
            else:
                template['company'][key_in_template] = value

    if 'educationRequirements' in data.keys() and isinstance(data['educationRequirements'], dict):
        template['education']['required_credential'] = data['educationRequirements']['credentialCategory']
    if 'experienceRequirements' in data.keys() and isinstance(data['experienceRequirements'], dict):
        template['experience']['months_of_experience'] = data['experienceRequirements']['monthsOfExperience']
    template['experience']['seniority_level'] = "I don't know"

    for key_in_template, key_in_data in location_mapping.items():
        if key_in_data in data.keys():
            keys = key_in_data.split('.')
            value = data
            for key in keys:
                if key in value:
                    value = value[key]
                else:
                    break
            else:
                template['location'][key_in_template] = value

    for key_in_template, key_in_data in salary_mapping.items():
        if key_in_data in data.keys():
            keys = key_in_data.split('.')
            value = data
            for key in keys:
                if key in value:
                    value = value[key]
                else:
                    break
            else:
                template['salary'][key_in_template] = value

template

{'job': {'title': 'Software Engineer, Early Career',
  'industry': 'Information Services,Internet',
  'description': "<strong>Please complete your application before August 13, 2021.</strong> By applying to this position, you will be considered for all potential Software Engineering opportunities. Roles may vary by location and will be discussed in more detail with your recruiter after completion of onsite interviews.<br><br>This program is intended for candidates with 1 to 3 years of experience. If you have less than 1 year of experience, please check the students careers site for internships and university graduate opportunities. If you have more than 3 years of experience, please check our industry openings.<br><br>Note: By applying to this position you will have an opportunity to share your preferred working location for this position from the following: <strong>Mountain View, CA, USA; Atlanta, GA, USA; Austin, TX, USA; Boulder, CO, USA; Cambridge, MA, USA; Chicago, IL, USA; Irvine

In [51]:
def extract_nested_value(data, keys_path):
    for key in keys_path:
        if key in data:
            data = data[key]
        else:
            return None
    return data

job_mapping = {
    'date_posted': 'datePosted',
    'description': 'description',
    'employment_type': 'employmentType',
    'industry': 'industry',
    'title': 'title'
}
location_mapping = {
    'country': 'jobLocation.address.addressCountry',
    'latitude': 'jobLocation.latitude',
    'locality': 'jobLocation.address.addressLocality',
    'longitude': 'jobLocation.longitude',
    'postal_code': 'jobLocation.address.postalCode',
    'region': 'jobLocation.address.addressRegion',
    'street_address': 'jobLocation.address.streetAddress',
}
company_mapping = {
    'link': 'hiringOrganization.sameAs',
    'name': 'hiringOrganization.name',
}
salary_mapping = {
    'currency': 'estimatedSalary.currency',
    'max_value': 'estimatedSalary.value.maxValue',
    'min_value': 'estimatedSalary.value.minValue',
    'unit': 'estimatedSalary.value.unitText',
}
transformed_data = []
for data in data_as_json:
    fill_template = template
    for key_in_template, key_in_data in job_mapping.items():
        fill_template['job'][key_in_template] = data.get(key_in_data, None)

    for key_in_template, key_in_data in company_mapping.items():
        fill_template['company'][key_in_template] = extract_nested_value(data, key_in_data.split('.'))

    fill_template['education']['required_credential'] = extract_nested_value(data, 'educationRequirements.credentialCategory'.split('.'))

    if 'experienceRequirements' in data and isinstance(data['experienceRequirements'], dict):
        fill_template['experience']['months_of_experience'] = data['experienceRequirements'].get('monthsOfExperience', None)
    fill_template['experience']['seniority_level'] = "I don't know"

    for key_in_template, key_in_data in location_mapping.items():
        fill_template['location'][key_in_template] = extract_nested_value(data, key_in_data.split('.'))

    for key_in_template, key_in_data in salary_mapping.items():
        fill_template['salary'][key_in_template] = extract_nested_value(data, key_in_data.split('.'))
    
    transformed_data.append(template)

In [55]:
template

{'job': {'title': 'Software Engineer, Early Career',
  'industry': 'Information Services,Internet',
  'description': "<strong>Please complete your application before August 13, 2021.</strong> By applying to this position, you will be considered for all potential Software Engineering opportunities. Roles may vary by location and will be discussed in more detail with your recruiter after completion of onsite interviews.<br><br>This program is intended for candidates with 1 to 3 years of experience. If you have less than 1 year of experience, please check the students careers site for internships and university graduate opportunities. If you have more than 3 years of experience, please check our industry openings.<br><br>Note: By applying to this position you will have an opportunity to share your preferred working location for this position from the following: <strong>Mountain View, CA, USA; Atlanta, GA, USA; Austin, TX, USA; Boulder, CO, USA; Cambridge, MA, USA; Chicago, IL, USA; Irvine

In [56]:
mapping = {
    'job': {
        'date_posted': 'datePosted',
        'description': 'description',
        'employment_type': 'employmentType',
        'industry': 'industry',
        'title': 'title',
    },
    'location': {
        'country': 'jobLocation.address.addressCountry',
        'latitude': 'jobLocation.latitude',
        'locality': 'jobLocation.address.addressLocality',
        'longitude': 'jobLocation.longitude',
        'postal_code': 'jobLocation.address.postalCode',
        'region': 'jobLocation.address.addressRegion',
        'street_address': 'jobLocation.address.streetAddress',
    },
    'company': {
        'link': 'hiringOrganization.sameAs',
        'name': 'hiringOrganization.name',
    },
    'salary': {
        'currency': 'estimatedSalary.currency',
        'max_value': 'estimatedSalary.value.maxValue',
        'min_value': 'estimatedSalary.value.minValue',
        'unit': 'estimatedSalary.value.unitText',
    },
    'education': {
        'required_credential': 'educationRequirements.credentialCategory',
    },
    'experience': {
        'months_of_experience': 'experienceRequirements.monthsOfExperience',
        'seniority_level': "I don't know",
    },
}
transformed_data = []
for data in data_as_json:
    template = {
        "job": {
            "title": None,
            "industry": None,
            "description": None,
            "employment_type": None,
            "date_posted": None,
        },
        "company": {
            "name": None,
            "link": None,
        },
        "education": {
            "required_credential": None,
        },
        "experience": {
            "months_of_experience": None,
            "seniority_level": None,
        },
        "salary": {
            "currency": None,
            "min_value": None,
            "max_value": None,
            "unit": None,
        },
        "location": {
            "country": None,
            "locality": None,
            "region": None,
            "postal_code": None,
            "street_address": None,
            "latitude": None,
            "longitude": None,
        },
    }
    for section, section_mapping in mapping.items():
        for key_in_template, key_in_data in section_mapping.items():
            keys = key_in_data.split('.')
            value = data
            for key in keys:
                if key in value:
                    value = value[key]
                else:
                    break
            else:
                template[section][key_in_template] = value
    transformed_data.append(template)


In [54]:
len(data_as_json)

7685

In [49]:
data_as_json[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-08-25T17:45:15.000Z',
 'description': "<p><strong><u>ABOUT US:</u></strong></p><p>As the leader of rehabilitative care, Encompass Health offers both facility-based and home-based patient care through its national network of rehabilitation hospitals, home health agencies, and hospice agencies that spans 42 states and Puerto Rico. Setting the standard for providing excellent care, Encompass Health has earned its place among Modern Healthcare’s “Best Places to Work in Healthcare” and the Fortune “World’s Most Admired Companies” for 2021.</p><p><br></p><p><strong><u>POSITION PURPOSE:</u></strong></p><p>The&nbsp;<strong>HRIS Security &amp; Data Maintenance Analyst</strong>&nbsp;plays an integral role in HR Systems security definition and is responsible for defining, setting up, maintaining and accurately documenting security access for all HR systems. The HRIS Security &amp; Data Maintenance Analyst also oversees

In [57]:
transformed_data[0]

{'job': {'title': 'HRIS Security & Data Maintenance Analyst',
  'industry': 'Hospital & Health Care',
  'description': "<p><strong><u>ABOUT US:</u></strong></p><p>As the leader of rehabilitative care, Encompass Health offers both facility-based and home-based patient care through its national network of rehabilitation hospitals, home health agencies, and hospice agencies that spans 42 states and Puerto Rico. Setting the standard for providing excellent care, Encompass Health has earned its place among Modern Healthcare’s “Best Places to Work in Healthcare” and the Fortune “World’s Most Admired Companies” for 2021.</p><p><br></p><p><strong><u>POSITION PURPOSE:</u></strong></p><p>The&nbsp;<strong>HRIS Security &amp; Data Maintenance Analyst</strong>&nbsp;plays an integral role in HR Systems security definition and is responsible for defining, setting up, maintaining and accurately documenting security access for all HR systems. The HRIS Security &amp; Data Maintenance Analyst also overse

In [ ]:
"""<p><strong><u>ABOUT US:</u></strong></p><p>As the leader of rehabilitative care, Encompass Health offers both facility-based and home-based patient care through its 
national network of rehabilitation hospitals, home health agencies, and hospice agencies that spans 42 states and Puerto Rico. Setting the standard for providing excellent 
care, Encompass Health has earned its place among Modern Healthcare’s “Best Places to Work in Healthcare” and the Fortune “World’s Most Admired Companies” for 2021.
</p><p><br></p><p><strong><u>POSITION PURPOSE:</u></strong></p><p>The&nbsp;<strong>HRIS Security &amp; Data Maintenance Analyst</strong>&nbsp;plays an integral role in HR 
Systems security definition and is responsible for defining, setting up, maintaining and accurately documenting security access for all HR systems. The HRIS Security &amp; 
Data Maintenance Analyst also oversees application support including table setup and maintenance, data integrity, and auditing of data. Additionally, this role will assist 
with the overall administrative support of existing HR systems including upgrades, conversions, and problem resolution.</p><p><br></p><p><strong><u>RESPONSIBILITES &amp; 
TASKS:</u></strong></p><p>SECURITY &amp; AUDITS:</p><ul><li>Defines and sets up security roles and permissions for HR Systems.</li><li>Defines and maintains dynamic role 
and permission assignment for auto provisioning and de-provisioning.</li><li>Creates and maintains a crosswalk that outlines role permissions and assigned titles.</li><li>
Creates queries/reports that extract role definitions and permissions for auditing purposes.</li><li>Acts as security administrator by processing access requests to HR 
applications and systems.</li><li>Ensures appropriate approvals are received for any ad hoc requests.</li><li>Serves as the SME that partners with various business areas to 
provide guidance on best practices and current HR security protocols.</li><li>Performs periodic audits of HR system user security/access.</li><li>Validates current company 
structure against employee job movement to maintain continuous access accuracy.</li><li>Looks for opportunities to increase efficiencies for common requests.</li><li>Supports 
employees/end users with HR system access issues.</li><li>Adheres to department’s SLA for processing requests.</li></ul><p><br></p><p>TABLE MAINTENANCE, UPLOADS &amp; DATA 
INTEGRITY AUDITS:</p><ul><li>Responsible for mass data uploads, coordinating with HR contacts to ensure accuracy in the process.</li><li>Performs data structure and integrity 
audits on employee data mass upload requests.</li><li>Seeks to minimize errors and determine impacts to downstream systems/processes, as well as impacts to employees due to 
requested changes/movement.</li><li>Updates system fields and tables associated with business changes and operations.</li><li>Maintains tables and security changes within HR 
Systems.</li><li>Proactively utilizes PeopleSoft Query, OBI Reporting or SQL tools to identify data integrity issues.</li><li>Determines the appropriate approach to ensure 
employee and company structure data elements are updated correctly.</li><li>Conducts data audits to identify potential non-compliance. I.e. personal data audits…duplicate 
NID, DOB out of guidelines.</li><li>Partners with internal audit to research and analyze potential data discrepancies.</li><li>Partners with ITG to implement automated access 
audit processes.</li><li>Participates in new projects through testing/implementation as it relates to access and security.</li><li>Completes mandatory training and courses 
required by completion date.</li><li>Reports questionable situations, concerns, complaints or harassment immediately.</li><li>Conducts audits of system process completions 
and reports out to business owners. i.e. Onboarding not complete.</li><li>Administers duplicate processing in ATS and other HR systems.</li><li>Ensures compliance regarding 
HR information sharing, management, organization, hierarchy determination, retrieval, accessibility, dissemination, sharing, and quality control.
</li></ul><p><br></p><p><strong><u>Qualifications:</u></strong></p><p>Minimum Qualifications:</p><ul><li>Bachelor's degree preferred. Associate's degree with 4 years of 
related experience may be substituted for Bachelor's degree.</li><li>Four or more years of experience required in related field (i.e. Human Resources Operations, HRIS, HR 
Technology, IT, etc.).</li><li>Strong knowledge of Microsoft Word, Excel, and PowerPoint. Advanced Excel skills preferred. Strong analytic and diagnostic skills as well as 
a data-driven and metrics-driven approach.</li><li>Minimum 3 years of experience with PeopleSoft version 9.2 or higher preferred.</li><li>Experience with workflow analysis, 
business process analysis, finding improvement areas, and use of business modeling tools.</li><li>Experience with security and auditing processes.</li></ul><p><br></p><p>
Machines, Equipment Used:</p><ul><li>General office equipment such as computer/laptop, telephone, copy/fax machine, calculator, scanner, etc.</li><li>Microsoft Office 
software, to include Outlook, Word, PowerPoint, and Excel.</li></ul><p><br></p><p>Physical Requirements:</p><ul><li>Visual acuity, speech recognition, speech clarity.
</li><li>Ability to lift, lower, push, pull, and retrieve objects weighing a minimum of 15 pounds. Reasonable assistance may be requested when lifting, pushing, and/or 
pulling exceeds these minimum requirements.</li></ul><p><br></p><p>Skills and Abilities:</p><ul><li>Oral communication, written communication, fluency in English, active 
listening.</li><li>Information ordering, deductive reasoning, social perceptiveness, time management, critical thinking.</li><li>Ability to coordinate, analyze, observe, 
make decisions, and meet deadlines in a detail-oriented manner.</li><li>Ability to work independently without continuous supervision.</li><li>Ability to understand and 
react quickly to changing priorities.</li><li>Demonstrated accuracy and thoroughness while meeting customer needs in a timely manner.</li></ul><p><br></p><p>Environmental 
Conditions:</p><ul><li>Indoor, temperature controlled, smoke-free environment.</li><li>Handicapped accessible.</li><li>May work under stressful circumstances at times.
</li></ul><p><br></p><p>Proficiency or Productivity Standards:</p><ul><li>Has regular, reliable, and predictable attendance and punctuality.</li><li>Adheres to dress code 
including wearing ID badge.</li><li>Adheres to Standards of Business Conduct.</li><li>May be required to work weekdays and/or weekends, evenings and/or night shifts if needed 
to meet deadlines.</li><li>May be required to work on religious and/or legal holidays on scheduled days/shifts.</li><li>May be required to perform other duties as assigned 
by supervisor.</li><li>This position will support cultural diversity by promoting and maintaining an inclusive work environment and culture that is respectful and accepting 
of diversity.</li></ul><p></p>",
 """

In [14]:
data_as_json[2]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-07-27T11:03:36.000Z',
 'description': 'Tinder brings people together. With tens of millions of users and a presence in every country on earth, our reach is expansive and rapidly growing.<br><br>The Analytics team powers data-informed decision-making at Tinder. We are looking to add a hardworking, creative Data Scientist to our growing team. As a Data Scientist on Analytics, you will partner with multiple teams - including Product, Marketing, Engineering, and Finance - to improve our extraordinary platform. You will use this comprehensive dataset to develop experiments, forecasting models, automated reports, and various exploratory insights to guide our business decisions.<br><br>An effective Data Scientist at Tinder must have a passion and aptitude for turning sophisticated problems into concrete hypotheses that can be answered through thorough data analysis and A/B testing. They must also be able to dissemi

In [ ]:
"""Tinder brings people together. With tens of millions of users and a presence in every country on earth, our reach is expansive and rapidly growing.
<br><br>The Analytics team powers data-informed decision-making at Tinder. We are looking to add a hardworking, creative Data Scientist to our growing team.
As a Data Scientist on Analytics, you will partner with multiple teams - including Product, Marketing, Engineering, and Finance - to improve our extraordinary platform.
You will use this comprehensive dataset to develop experiments, forecasting models, automated reports, and various exploratory insights to guide our business decisions.
<br><br>An effective Data Scientist at Tinder must have a passion and aptitude for turning sophisticated problems into concrete hypotheses that can be answered through 
thorough data analysis and A/B testing. They must also be able to disseminate their takeaways and recommendations to a variety of partners within our organization via 
presentations, reports, and dashboards. In this role, your insights will influence both the performance of our business and the experience of our users.<br><br><strong>
Responsibilities:<br></strong><li>Be a leader with a point of view for the organization, using data to constantly challenge and reshape our current understanding of 
customer behavior and business performance</li><li>Derive and communicate data-driven recommendations regarding both the prioritization of potential product/marketing
initiatives and performance of current initiatives</li><li>Define and operationalize the detailed tracking of company-wide, team-specific, and product-specific 
performance metrics via rollup tables, dashboards, and automated reporting</li><li>Design and analyze experiments to rigorously test a specific hypothesis</li><li>
Find the “story” in the data and readily share insights across the organization<br><br></li><strong>Skills Required:<br></strong><li>No formal industry experience is 
required - we are looking for creative, tenacious problem solvers who are passionate about our product and excited to work with our rich, complex dataset</li><li>Ability
to develop a nuanced business understanding, testable hypotheses, and meaningful findings from data</li><li>Eagerness to identify and explore new product/marketing 
opportunities, potential improvements to our data infrastructure, and unanswered questions about customer behavior</li><li>Outstanding written and verbal communication
skills, and a willingness to collaborate with business partners</li><li>Comfort using descriptive and inferential statistics</li><li>Experience with SQL and data 
visualization tools (e.g. Tableau or Mode) is highly recommended</li><li>Familiarity with scripting languages, machine learning, advanced statistical techniques, 
and A/B testing is a plus<br><br></li><strong>As part of our team, you’ll enjoy:<br></strong><li> Working on a product that has an immediate impact on people’s 
lives all around the world</li><li> Collaborating with a team of creative, fun and driven colleagues</li><li> Comprehensive health coverage, competitive salary,
401(k) employer match, Employee Stock Purchase Plan (ESPP)</li><li> Other perks and wellness benefits like a fitness membership subsidy, paid concierge medical 
membership, pet insurance offerings, and a commuter subsidy</li><li> A robust Learning + Development offering including our MentorMatch program, access to a 
library of 6,000+ online learning courses through Udemy, and an annual L+D stipend</li><li> Access to mental health resources</li><li> Fertility preservation 
benefits</li><li> No Meeting Wednesdays and access to a wide range of product and service discounts through Perkspot</li><li> Charitable donations match up 
to $15,000 annually</li><li> Monthly and weekly interactive virtual events including Book Club, trivia with prizes and yoga workouts</li><li> The opportunity
to join six active Employee Resource Groups (ERGs)<br><br></li>At Tinder, we don’t just accept difference — we celebrate it, we support it, and we thrive 
on it for the benefit of our employees, our product, and our community. We strive to make our workplace an inclusive and diverse environment, giving people 
from all walks of life the opportunity to have a voice. We champion and encourage those who bring different perspectives, ideas, and creativity to join our 
team dedicated to bringing people together across the globe. Tinder is proud to be an equal opportunity workplace where we welcome all people regardless of 
sex, gender identity, race, ethnicity, disability, or other lived experience.',
 """

In [15]:
data_as_json[15]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-09-02T06:08:31.000Z',
 'description': 'The volunteer opportunity below is a request from an educator for you to interact with his or her classroom. The interaction is virtual so as long as you have a good internet connection and computer or device with a camera, you can participate from anywhere. The goal of the interaction is to bring real world application, professional experience and relevance to what the educator is teaching. Once you accept the request for one of the dates proposed by the educator, Nepris will provide support and next steps to help you prepare and make sure you are successful. You can also highlight this experience to strengthen the volunteer section of your LinkedIn profile.<br><br><strong>Volunteer Virtual Classroom Speaker (Meet a Software Engineer)<br><br></strong><strong><u>Description<br><br></u></strong>We are investigating potential career pathways. The goal of this conversation

In [16]:
data_as_json[200]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-08-16T11:25:14.000Z',
 'description': "<strong><u>About NCR<br><br></u></strong>NCR Corporation (NYSE: NCR) is a leading software- and services-led enterprise provider in the financial, retail and hospitality industries. NCR is headquartered in Atlanta, Georgia, with 36,000 employees globally. NCR is a trademark of NCR Corporation in the United States and other countries.<br><br>Software Engineer - Dot Net developer<br><br>Location- Addison, TX<br><br><strong><u>Position Summary &amp; Key Areas Of Responsibility<br><br></u></strong>Working on the NCR Payments Gateway Team by working with team of software engineers to provide new features and functionality to the existing and next generation applications<br><br>Supporting production by triaging and managing hot fix releases and defects throughout the development life cycle including testing and deployment.<br><br>Facilitate requirements refinement and design 

In [17]:
data_as_json[300]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-08-07T18:11:21.000Z',
 'description': "In this age of disruption, organizations need to navigate the future with confidence by tapping into the power of data analytics, robotics, and cognitive technologies such as Artificial Intelligence (AI). Our Strategy &amp; Analytics portfolio helps clients leverage rigorous analytical capabilities and a pragmatic mindset to solve the most complex of problems. By joining our team, you will play a key role in helping to our clients uncover hidden relationships from vast troves of data and transforming the Government and Public Services marketplace.<br><br>Work you'll do<br><br>The Data Scientist will join a team responsible for developing advanced analytics products; applying data visualization and statistical programming tools to enterprise data to advance and enable the key mission outcomes. In this role, they will support all phases of analytic work product developmen

In [9]:
template = {
            "job": {
                "title": "None",
                "industry": "None",
                "description": "None",
                "employment_type": "None",
                "date_posted": "None",
            },
            "company": {
                "name": "None",
                "link": "None",
            },
            "education": {
                "required_credential": "None",
            },
            "experience": {
                "months_of_experience": "None",
                "seniority_level": "None",
            },
            "salary": {
                "currency": "None",
                "min_value": "None",
                "max_value": "None",
                "unit": "None",
            },
            "location": {
                "country": "None",
                "locality": "None",
                "region": "None",
                "postal_code": "None",
                "street_address": "None",
                "latitude": "None",
                "longitude": "None",
            },
        }

In [10]:
columns = template.keys()

In [13]:
for col in columns:
    feilds = list(template[col].keys())
    values = ', '.join([f"'{template[col][feild]}'" for feild in feilds])
    feilds = ', '.join(feilds)

    print(f"INSERT INTO {col} ({feilds}) VALUES ({values});")


INSERT INTO job (title, industry, description, employment_type, date_posted) VALUES ('None', 'None', 'None', 'None', 'None');
INSERT INTO company (name, link) VALUES ('None', 'None');
INSERT INTO education (required_credential) VALUES ('None');
INSERT INTO experience (months_of_experience, seniority_level) VALUES ('None', 'None');
INSERT INTO salary (currency, min_value, max_value, unit) VALUES ('None', 'None', 'None', 'None');
INSERT INTO location (country, locality, region, postal_code, street_address, latitude, longitude) VALUES ('None', 'None', 'None', 'None', 'None', 'None', 'None');
